In [1]:
import os
os.chdir('../../')

In [2]:
import pandas as pd
import re
from google.oauth2 import service_account
from google.cloud import bigquery

In [3]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [4]:
SERVICE_ACCOUNT = 'private/bigquery-write-access.json'
DATASET = 'student_counts'
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT)

In [5]:
bqclient = bigquery.Client(project=credentials.project_id, credentials=credentials)

## Read Data

In [6]:
query_string = \
"""
SELECT * 
FROM `int-stud-mobility-drivers.datasets_for_modelling.students_by_year_countrly_all_predictors` 
"""
dataframe = bqclient.query(query_string).result().to_dataframe()

In [7]:
country_codes = {
    "Australia": 'aus', 
    'Canada': 'can', 
    'United Kingdom': 'uk',
    'United States of America': 'us',
    'New Zealand': 'nz'
}
destination_col = 'destination_code'
dataframe[destination_col] = dataframe.destination_country.apply(lambda x: country_codes[x])

In [8]:
#dataframe = dataframe[dataframe.source_country == 'China']
#dataframe = dataframe[dataframe.destination_country == 'Canada']

In [9]:
index_cols = ['source_country', 'year']
dst_cols = [col for col in dataframe.columns if col.startswith('dst')]
src_cols = [col for col in dataframe.columns if col.startswith('src')]

## Transform Destination Country Variables 

In [10]:
dest_index = index_cols + [destination_col]
dest_df = dataframe[dest_index + dst_cols].copy()
dest_df.set_index(dest_index, inplace=True)
dest_df.head()

dst_distance  dst_students_count  \
source_country year destination_code                                     
China          2017 aus                       7470              128498   
               2016 aus                       7470              112329   
               2015 aus                       7470               97387   
               2014 aus                       7470               90245   
               2011 aus                       7470               90175   

                                      dst_average_wage  dst_cost_of_living  \
source_country year destination_code                                         
China          2017 aus                        53507.0               80.66   
               2016 aus                        53972.0               78.45   
               2015 aus                        53946.0               99.32   
               2014 aus                        54392.0              108.51   
               2011 aus                        53906.0              107.83   

                                      dst_rent_cost_of_liv  dst_rent_index  \
source_country year destination_code                                         
China          2017 aus                              62.39           42.54   
               2016 aus                              60.43           40.95   
               2015 aus                              74.96           49.47   
               2014 aus                              83.95           57.97   
               2011 aus                              87.43           51.03   

                                      dst_groceries_index  \
source_country year destination_code                        
China          2017 aus                             76.87   
               2016 aus                             74.68   
               2015 aus                             94.31   
               2014 aus                            105.03   
               2011 aus                            112.03   

                                      dst_purchasing_power_index  \
source_country year destination_code                               
China          2017 aus                                   101.94   
               2016 aus                                   147.25   
               2015 aus                                   110.39   
               2014 aus                                   104.20   
               2011 aus                                   117.10   

                                      dst_resraurant_price_index  \
source_country year destination_code                               
China          2017 aus                                    77.12   
               2016 aus                                    74.85   
               2015 aus                                    90.63   
               2014 aus                                    97.44   
               2011 aus                                    88.67   

                                      dst_rate_of_one_usd  dst_qoe_top20  \
source_country year destination_code                                       
China          2017 aus                          1.304758            0.0   
               2016 aus                          1.345214            0.0   
               2015 aus                          1.331090            0.0   
               2014 aus                          1.109363            0.0   
               2011 aus                          0.969463            0.0   

                                      dst_qoe_top100  dst_qoe_top200  \
source_country year destination_code                                   
China          2017 aus                          6.0            10.0   
               2016 aus                          6.0             8.0   
               2015 aus                          4.0             8.0   
               2014 aus                          4.0             8.0   
               2011 aus                          4.0             7.0   

     

In [11]:
dest_df = dest_df.unstack(level=destination_col)
dest_df.head()

dst_distance                                    \
destination_code             aus     can       nz      uk       us   
source_country year                                                  
China          1998       7470.0  9380.0  11160.0  7775.0      NaN   
               1999       7470.0  9380.0  11160.0  7775.0  11640.0   
               2000       7470.0  9380.0  11160.0  7775.0  11640.0   
               2001          NaN  9380.0  11160.0  7775.0  11640.0   
               2002       7470.0  9380.0  11160.0  7775.0  11640.0   

                    dst_students_count                                     \
destination_code                   aus      can      nz       uk       us   
source_country year                                                         
China          1998             4132.0   2820.0    78.0   2877.0      NaN   
               1999             4578.0   3489.0   247.0   4250.0  46949.0   
               2000             5008.0   4701.0  1133.0   6158.0  50281.0   
               2001                NaN   6972.0  3338.0  10388.0  51986.0   
               2002            17343.0  10176.0  8481.0  17483.0  63211.0   

                    dst_average_wage                                      \
destination_code                 aus      can       nz       uk       us   
source_country year                                                        
China          1998              NaN      NaN      NaN      NaN      NaN   
               1999              NaN      NaN      NaN      NaN      NaN   
               2000          45459.0  39380.0  31806.0  38908.0  53904.0   
               2001              NaN  39218.0  32391.0  40840.0  54360.0   
               2002          46270.0  38800.0  33005.0  41517.0  54788.0   

                    dst_cost_of_living                 dst_rent_cost_of_liv  \
destination_code                   aus can  nz  uk  us                  aus   
source_country year                                                           
China          1998                NaN NaN NaN NaN NaN                  NaN   
               1999                NaN NaN NaN NaN NaN                  NaN   
               2000                NaN NaN NaN NaN NaN                  NaN   
               2001                NaN NaN NaN NaN NaN                  NaN   
               2002                NaN NaN NaN NaN NaN                  NaN   

                                    dst_rent_index                  \
destination_code    can  nz  uk  us            aus can  nz  uk  us   
source_country year                                                  
China          1998 NaN NaN NaN NaN            NaN NaN NaN NaN NaN   
               1999 NaN NaN NaN NaN            NaN NaN NaN NaN NaN   
               2000 NaN NaN NaN NaN            NaN NaN NaN NaN NaN   
               2001 NaN NaN NaN NaN            NaN NaN NaN NaN NaN   
               2002 NaN NaN NaN NaN            NaN NaN NaN NaN NaN   

                    dst_groceries_index                  \
destination_code                    aus can  nz  uk  us   
source_country year                                       
China          1998                 NaN NaN NaN NaN NaN   
               1999                 NaN NaN NaN NaN NaN   
               2000                 NaN NaN NaN NaN NaN   
               2001                 NaN NaN NaN NaN NaN   
               2002                 NaN NaN NaN NaN NaN   

                    dst_purchasing_power_index                  \
destination_code                           aus can  nz  uk  us   
source_country year                                              
China          1998                        NaN NaN NaN NaN NaN   
               1999                        NaN NaN NaN NaN NaN   
               2000                        NaN NaN NaN NaN NaN   
               2001                        NaN NaN NaN NaN NaN   
               2002                        NaN NaN NaN NaN NaN   

                    dst_resraurant_pr

In [12]:
dest_df.columns = ['{}_{}'.format(var, code) for var, code in dest_df.columns.values]

In [13]:
dest_df.head()

dst_distance_aus  dst_distance_can  dst_distance_nz  \
source_country year                                                        
China          1998            7470.0            9380.0          11160.0   
               1999            7470.0            9380.0          11160.0   
               2000            7470.0            9380.0          11160.0   
               2001               NaN            9380.0          11160.0   
               2002            7470.0            9380.0          11160.0   

                     dst_distance_uk  dst_distance_us  dst_students_count_aus  \
source_country year                                                             
China          1998           7775.0              NaN                  4132.0   
               1999           7775.0          11640.0                  4578.0   
               2000           7775.0          11640.0                  5008.0   
               2001           7775.0          11640.0                     NaN   
               2002           7775.0          11640.0                 17343.0   

                     dst_students_count_can  dst_students_count_nz  \
source_country year                                                  
China          1998                  2820.0                   78.0   
               1999                  3489.0                  247.0   
               2000                  4701.0                 1133.0   
               2001                  6972.0                 3338.0   
               2002                 10176.0                 8481.0   

                     dst_students_count_uk  dst_students_count_us  \
source_country year                                                 
China          1998                 2877.0                    NaN   
               1999                 4250.0                46949.0   
               2000                 6158.0                50281.0   
               2001                10388.0                51986.0   
               2002                17483.0                63211.0   

                     dst_average_wage_aus  dst_average_wage_can  \
source_country year                                               
China          1998                   NaN                   NaN   
               1999                   NaN                   NaN   
               2000               45459.0               39380.0   
               2001                   NaN               39218.0   
               2002               46270.0               38800.0   

                     dst_average_wage_nz  dst_average_wage_uk  \
source_country year                                             
China          1998                  NaN                  NaN   
               1999                  NaN                  NaN   
               2000              31806.0              38908.0   
               2001              32391.0              40840.0   
               2002              33005.0              41517.0   

                     dst_average_wage_us  dst_cost_of_living_aus  \
source_country year                                                
China          1998                  NaN                     NaN   
               1999                  NaN                     NaN   
               2000              53904.0                     NaN   
               2001              54360.0                     NaN   
               2002              54788.0                     NaN   

                     dst_cost_of_living_can  dst_cost_of_living_nz  \
source_country year                                                  
China          1998                     NaN                    NaN   
               1999                     NaN                    NaN   
               2000                     NaN                    NaN   
               2001                     NaN                    NaN   
               2002                     NaN                    NaN   

                     dst_cost_of_living_uk  dst_cos

## Source Country Variables

In [14]:
source_df = dataframe[index_cols + src_cols].copy().drop_duplicates()
source_df.set_index(index_cols, inplace=True)
source_df.head()

src_population  src_qoe_top20  src_qoe_top100  \
source_country year                                                  
China          2017      1386395000            0.0             2.0   
               2016      1378665000            0.0             2.0   
               2015      1371220000            0.0             0.0   
               2014      1364270000            0.0             0.0   
               2011      1344130000            0.0             0.0   

                     src_qoe_top200  src_qoe_top500  src_gni       src_gdp  \
source_country year                                                          
China          2017            13.0            57.0   8630.0  1.210000e+13   
               2016            12.0            54.0   8210.0  1.110000e+13   
               2015            10.0            44.0   7910.0  1.100000e+13   
               2014             9.0            44.0   7500.0  1.040000e+13   
               2011             3.0            35.0   5050.0  7.550000e+12   

                     src_rate_of_one_usd  src_tertiary_enrolment  
source_country year                                               
China          2017             6.758755               51.006741  
               2016             6.644478               48.441051  
               2015             6.227489               45.350719  
               2014             6.143434               41.277729  
               2011             6.461461               25.292101

## Join source and destination predictors

In [15]:
dataset_df = pd.concat([source_df, dest_df], sort=False, axis=1)
dataset_df

src_population  src_qoe_top20  src_qoe_top100  \
source_country year                                                  
China          1998      1241935000            NaN             NaN   
               1999      1252735000            NaN             NaN   
               2000      1262645000            NaN             NaN   
               2001      1271850000            NaN             NaN   
               2002      1280400000            NaN             NaN   
               2003      1288400000            NaN             NaN   
               2004      1296075000            0.0             0.0   
               2005      1303720000            0.0             0.0   
               2006      1311020000            0.0             0.0   
               2007      1317885000            0.0             0.0   
               2008      1324655000            0.0             0.0   
               2009      1331260000            0.0             0.0   
               2010      1337705000            0.0             0.0   
               2011      1344130000            0.0             0.0   
               2012      1350695000            0.0             0.0   
               2013      1357380000            0.0             0.0   
               2014      1364270000            0.0             0.0   
               2015      1371220000            0.0             0.0   
               2016      1378665000            0.0             2.0   
               2017      1386395000            0.0             2.0   
India          1998      1019483581            NaN             NaN   
               1999      1038058156            NaN             NaN   
               2000      1056575549            NaN             NaN   
               2001      1075000085            NaN             NaN   
               2002      1093317189            NaN             NaN   
               2003      1111523144            NaN             NaN   
               2004      1129623456            0.0             0.0   
               2005      1147609927            0.0             0.0   
               2006      1165486291            0.0             0.0   
               2007      1183209472            0.0             0.0   
               2008      1200669765            0.0             0.0   
               2009      1217726215            0.0             0.0   
               2010      1234281170            0.0             0.0   
               2011      1250288729            0.0             0.0   
               2012      1265782790            0.0             0.0   
               2013      1280846129            0.0             0.0   
               2014      1295604184            0.0             0.0   
               2015      1310152403            0.0             0.0   
               2016      1324509589            0.0             0.0   
               2017      1338658835            0.0             0.0   
Indonesia      1998       205724592            NaN             NaN   
               1999       208615169            NaN             NaN   
               2000       211513823            NaN             NaN   
               2001       214427417            NaN             NaN   
               2002       217357793            NaN             NaN   
               2003       220309469            NaN             NaN   
               2004       223285676            NaN             NaN   
               2005       226289470            NaN             NaN   
               2006       229318262            NaN             NaN   
               2007       232374245            NaN             NaN   
               2008       235469762            NaN             NaN   
               2009       238620563            NaN             NaN   
               2010       241834215            NaN             NaN   
               2011       245116206            NaN             NaN   
               2012       248452413            NaN             NaN   
               2013       25180

In [17]:
table_full_name = '{}.{}'.format('datasets_for_modelling', 'students_by_year_countrly_transformed')
dataset_df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:06,  6.34s/it]


In [16]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 80 entries, (China, 1998) to (Malaysia, 2017)
Data columns (total 79 columns):
src_population                    80 non-null int64
src_qoe_top20                     35 non-null float64
src_qoe_top100                    35 non-null float64
src_qoe_top200                    35 non-null float64
src_qoe_top500                    35 non-null float64
src_gni                           80 non-null float64
src_gdp                           80 non-null float64
src_rate_of_one_usd               80 non-null float64
src_tertiary_enrolment            76 non-null float64
dst_distance_aus                  76 non-null float64
dst_distance_can                  76 non-null float64
dst_distance_nz                   76 non-null float64
dst_distance_uk                   76 non-null float64
dst_distance_us                   72 non-null float64
dst_students_count_aus            76 non-null float64
dst_students_count_can            76 non-null float64
dst_stude